## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,Mudkhed,IN,19.1667,77.5167,84.24,62,65,8.50,2022-07-31 05:58:36
1,1,Vallenar,CL,-28.5708,-70.7581,54.61,79,0,1.59,2022-07-31 05:58:37
2,2,Saint-Ambroise,CA,48.5501,-71.3324,66.74,72,86,8.05,2022-07-31 05:58:37
3,3,Bilma,NE,18.6853,12.9164,86.97,40,59,9.51,2022-07-31 05:58:38
4,4,Pinsk,BY,52.1229,26.0951,60.87,95,100,11.50,2022-07-31 05:55:23


In [8]:
#dropping city_id column
city_data_df.pop("City_ID")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Mudkhed,IN,19.1667,77.5167,84.24,62,65,8.50,2022-07-31 05:58:36
1,Vallenar,CL,-28.5708,-70.7581,54.61,79,0,1.59,2022-07-31 05:58:37
2,Saint-Ambroise,CA,48.5501,-71.3324,66.74,72,86,8.05,2022-07-31 05:58:37
3,Bilma,NE,18.6853,12.9164,86.97,40,59,9.51,2022-07-31 05:58:38
4,Pinsk,BY,52.1229,26.0951,60.87,95,100,11.50,2022-07-31 05:55:23


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 85


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Mudkhed,IN,19.1667,77.5167,84.24,62,65,8.50,2022-07-31 05:58:36
1,Vallenar,CL,-28.5708,-70.7581,54.61,79,0,1.59,2022-07-31 05:58:37
2,Saint-Ambroise,CA,48.5501,-71.3324,66.74,72,86,8.05,2022-07-31 05:58:37
4,Pinsk,BY,52.1229,26.0951,60.87,95,100,11.50,2022-07-31 05:55:23
6,Saquena,PE,-4.6667,-73.5167,74.34,87,81,2.13,2022-07-31 05:58:39
7,Luderitz,NaN,-26.6481,15.1594,50.74,70,0,16.04,2022-07-31 05:58:40
8,Alyangula,AU,-13.8483,136.4192,77.13,66,64,4.65,2022-07-31 05:58:40
10,Yellowknife,CA,62.4560,-114.3525,52.32,82,20,5.75,2022-07-31 05:58:41
11,Gari,RU,59.4307,62.3497,75.34,56,99,6.71,2022-07-31 05:58:42
13,Asau,RO,46.4333,26.4000,65.32,95,100,3.06,2022-07-31 05:58:43


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City          0
Country       4
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Date          0
dtype: int64

In [29]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [30]:
clean_df.count()

City          536
Country       536
Lat           536
Lng           536
Max Temp      536
Humidity      536
Cloudiness    536
Wind Speed    536
Date          536
dtype: int64

In [32]:
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mudkhed,IN,84.24,19.1667,77.5167,
1,Vallenar,CL,54.61,-28.5708,-70.7581,
2,Saint-Ambroise,CA,66.74,48.5501,-71.3324,
4,Pinsk,BY,60.87,52.1229,26.0951,
6,Saquena,PE,74.34,-4.6667,-73.5167,
8,Alyangula,AU,77.13,-13.8483,136.4192,
10,Yellowknife,CA,52.32,62.4560,-114.3525,
11,Gari,RU,75.34,59.4307,62.3497,
13,Asau,RO,65.32,46.4333,26.4000,
14,Nokaneng,BW,57.06,-19.6667,22.2667,


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [42]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City          536
Country       536
Max Temp      536
Lat           536
Lng           536
Hotel Name    536
dtype: int64

In [43]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [46]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))